In [2]:
import numpy as np
import cx_Oracle
import pandas as pd

In [3]:
dsn_tns = cx_Oracle.makedsn('cs322-db.epfl.ch', '1521', sid='ORCLCDB')
conn = cx_Oracle.connect(user=r'C##DB2019_G04', password='DB2019_G04', dsn=dsn_tns)

In [62]:
table_set = ['host', 'listing', 'neighborhood', 'amenity_equipment', 'bed_type',
            'calendar', 'cancellation_policy', 'city', 'country', 'host_verifications',
            'is_equiped_with', 'property_type', 'reserved_on' , 'response_time',
            'reviewer', 'reviews', 'room_type', 'verifies']

dsn_tns = cx_Oracle.makedsn('cs322-db.epfl.ch', '1521', sid='ORCLCDB')
conn = cx_Oracle.connect(user=r'C##DB2019_G04', password='DB2019_G04', dsn=dsn_tns)
value = 'abc'
columns_list = []
for i in table_set:
    cmd = 'SELECT * FROM ' + i + ' WHERE 1 = 2'
    c = conn.cursor()
    c.execute(cmd)
    columns = [row[0] for row in c.description]
    columns_list.append(columns)

print(columns_list)

'''string_prefices = []
for i in range(len(table_set)):
    string_where = ' WHERE '
    for j in range(len(columns_list[i])):
        string_where += ' REGEXP_LIKE(' + str(columns_list[i][j]) + ', \'' + str(value) + '\', \'im\')'  
        if j < (len(columns_list[i]) - 1):
            string_where += ' OR '
    string_prefix = 'SELECT * FROM '+ table_set[i] + string_where + 'FETCH FIRST 1 ROWS ONLY'

    string_prefices.append(string_prefix)

results = []

#e = conn.cursor()
#e.execute('ALTER TABLE reviews MODIFY ( comments CLOB )')
for i in string_prefices:
    c = conn.cursor()
    c.execute(i)   
    df = pd.DataFrame.from_records(list(c))
    column_names = [row[0] for row in c.description]
    columns = [{"id" : i, "name" : column_names[i]} for i in range(len(column_names))]
    results.append(df.to_dict('records'))
    results.append(columns)'''

[['HOST_ID', 'HOST_URL', 'HOST_NAME', 'HOST_SINCE', 'HOST_ABOUT', 'HOST_RESPONSE_RATE', 'HOST_THUMBNAIL_URL', 'HOST_PICTURE_URL', 'NEIGHBORHOOD_ID', 'RESPONSE_TIME_ID'], ['LISTING_ID', 'LISTING_URL', 'LISTING_NAME', 'LISTING_PICTURE_URL', 'DAILY_PRICE', 'WEEKLY_PRICE', 'MONTHLY_PRICE', 'SECURITY_DEPOSIT', 'CLEANING_FEE', 'ACCOMMODATES', 'GUESTS_INCLUDED', 'EXTRA_PEOPLE', 'MINIMUM_NIGHTS', 'MAXIMUM_NIGHTS', 'SUMMARY', 'SPACE', 'DESCRIPTION', 'NEIGHBORHOOD_OVERVIEW', 'NOTES', 'TRANSIT', 'LISTING_ACCESS', 'INTERACTION', 'BATHROOMS', 'BEDROOMS', 'BEDS', 'SQUARE_FEET', 'LATITUDE', 'LONGITUDE', 'IS_BUSINESS_TRAVEL_READY', 'REQUIRE_GPV', 'REQUIRE_GUEST_PROFILE_PICTURE', 'HOUSE_RULES', 'REVIEW_SCORES_VALUE', 'REVIEW_SCORES_CHECKIN', 'REVIEW_SCORES_CLEANLINESS', 'REVIEW_SCORES_ACCURACY', 'REVIEW_SCORES_RATING', 'REVIEW_SCORES_LOCATION', 'REVIEW_SCORES_COMMUNICATION', 'HOST_ID', 'NEIGHBORHOOD_ID', 'BEDTYPE_ID', 'ROOMTYPE_ID', 'CANCELLATION_POLICY_ID', 'PROPERTY_TYPE_ID'], ['NEIGHBORHOOD_ID', 'NE

'string_prefices = []\nfor i in range(len(table_set)):\n    string_where = \' WHERE \'\n    for j in range(len(columns_list[i])):\n        string_where += \' REGEXP_LIKE(\' + str(columns_list[i][j]) + \', \'\' + str(value) + \'\', \'im\')\'  \n        if j < (len(columns_list[i]) - 1):\n            string_where += \' OR \'\n    string_prefix = \'SELECT * FROM \'+ table_set[i] + string_where + \'FETCH FIRST 1 ROWS ONLY\'\n\n    string_prefices.append(string_prefix)\n\nresults = []\n\n#e = conn.cursor()\n#e.execute(\'ALTER TABLE reviews MODIFY ( comments CLOB )\')\nfor i in string_prefices:\n    c = conn.cursor()\n    c.execute(i)   \n    df = pd.DataFrame.from_records(list(c))\n    column_names = [row[0] for row in c.description]\n    columns = [{"id" : i, "name" : column_names[i]} for i in range(len(column_names))]\n    results.append(df.to_dict(\'records\'))\n    results.append(columns)'

In [11]:
print(type(results[3]))

<class 'list'>


In [4]:
cursor = conn.cursor()

In [61]:
pd.DataFrame.from_records(list(cursor.execute('SELECT * from HOST ORDER BY host_id FETCH FIRST 1 ROWS ONLY')))

,0,1,2,3,4,5,6,7,8,9
0,2217,https://www.airbnb.com/users/show/2217,ian,2008-08-18,believe in sharing economy.,96.0,https://a0.muscache.com/im/pictures/21428a22-4...,https://a0.muscache.com/im/pictures/21428a22-4...,74,1


In [14]:
dsn_tns = cx_Oracle.makedsn('cs322-db.epfl.ch', '1521', sid='ORCLCDB')
conn = cx_Oracle.connect(user=r'C##DB2019_G04', password='DB2019_G04', dsn=dsn_tns)
city = 'berlin'

string_prefix = ''' 
    SELECT up.neighborhood_name, up.neighborhood_id, up.count1 / down.count1 as ratio
FROM (SELECT NE.neighborhood_name, NE.neighborhood_id, COUNT(DISTINCT LI.listing_id) as count1
FROM CANCELLATION_POLICY CA, LISTING LI, NEIGHBORHOOD NE, CITY CI
WHERE LI.cancellation_policy_id = CA.cancellation_policy_id AND LI.neighborhood_id = NE.neighborhood_id AND NE.city_id = CI.city_id AND CA.cancellation_policy_name = 'strict_14_with_grace_period' AND CI.city_name = \'{0}\'
GROUP BY NE.neighborhood_name, NE.neighborhood_id) up,
(SELECT NE.neighborhood_name, NE.neighborhood_id, COUNT(DISTINCT LI.listing_id) as count1
FROM LISTING LI, NEIGHBORHOOD NE, CITY CI
WHERE LI.neighborhood_id = NE.neighborhood_id AND CI.CITY_NAME = \'{0}\'
GROUP BY NE.neighborhood_name, NE.neighborhood_id) down
WHERE up.neighborhood_id = down.neighborhood_id AND up.count1 >= 0.05 * down.count1
    '''.format(city)

#return string_prefix
c = conn.cursor()
c.execute(string_prefix)

df = pd.DataFrame.from_records(list(c))
columns = [
        {"id": 0, "name": "neighborhood_name"},
        {"id": 1, "name": "neighborhood_id"},
        {"id": 2, "name": "ratio"},
        ]


In [23]:
print(df.to_dict('records'))

[{0: 'k??penick', 1: 84, 2: 0.24390243902439024}, {0: 'charlottenburg', 1: 72, 2: 0.31511254019292606}, {0: 'lichterfelde', 1: 88, 2: 0.22077922077922077}, {0: 'siemensstadt', 1: 116, 2: 0.42857142857142855}, {0: 'kladow', 1: 134, 2: 0.21428571428571427}, {0: 'wittenau', 1: 93, 2: 0.17647058823529413}, {0: 'spandau', 1: 141, 2: 0.8148148148148148}, {0: 'potsdamer platz', 1: 80, 2: 0.2}, {0: 'schm??ckwitz', 1: 107, 2: 0.7142857142857143}, {0: 'nikolassee', 1: 106, 2: 0.14285714285714285}, {0: 'haselhorst', 1: 154, 2: 1.0}, {0: 'kreuzberg', 1: 66, 2: 0.2867647058823529}, {0: 'altglienicke', 1: 86, 2: 0.29411764705882354}, {0: 'hansaviertel', 1: 103, 2: 0.24324324324324326}, {0: 'zehlendorf', 1: 89, 2: 0.31666666666666665}, {0: 'alt-hohensch??nhausen', 1: 69, 2: 0.2222222222222222}, {0: 'lankwitz', 1: 124, 2: 0.21052631578947367}, {0: 'wannsee', 1: 117, 2: 0.3333333333333333}, {0: 'hermsdorf', 1: 102, 2: 0.5}, {0: 'heiligensee', 1: 145, 2: 0.3}, {0: 'marienfelde', 1: 136, 2: 0.14285714285

In [91]:
columns_list = []
for i in table_set:
    cmd = 'SELECT * FROM ' + i + ' WHERE 1 = 2'
    c = conn.cursor()
    c.execute(cmd)
    columns = [row[0] for row in c.description]
    columns_list.append(columns)

string_prefices = []
for i in range(len(table_set)):
    string_where = ' WHERE '
    for j in range(len(columns_list[i])):
        string_where += ' REGEXP_LIKE(' + str(columns_list[i][j]) + ', \'' + str(value) + '\', \'im\')'  
        if j < (len(columns_list[i]) - 1):
            string_where += ' OR '
    string_prefix = 'SELECT * FROM '+ table_set[i] + string_where + 'FETCH FIRST 1 ROWS ONLY'

    string_prefices.append(string_prefix)

results = []

for i in range(len(table_set)):
    string_prefix = string_prefices[i]
    c = conn.cursor()
    c.execute('SELECT * from REVIEWS FETCH FIRST 1 ROWS ONLY')
    l = list(c)
    if table_set[i] == 'host' :
        tu = list(l[0])
        tu.pop(0)
        l[0] = tuple(tu)
    if table_set[i] == 'reviews' :
        tu = list(l[0])
        tu.pop(3)
        l[0] = tuple(tu)
    df = pd.DataFrame.from_records(l)
    column_names = [row[0] for row in c.description if row[0] != 'HOST_ABOUT' and row[0] != 'COMMENTS']
    columns = [{"id" : i, "name" : column_names[i]} for i in range(len(column_names))]
    results.append(df.to_dict('records'))
    results.append(columns)

In [98]:
results = []

for i in range(len(table_set)):
    string_prefix = string_prefices[i]
    c = conn.cursor()
    c.execute(string_prefix)
    l = list(c)
    if table_set[i] == 'host' :
        tu = list(l[0])
        tu.pop(4)
        l[0] = tuple(tu)
    if table_set[i] == 'reviews' :
        tu = list(l[0])
        tu.pop(3)
        l[0] = tuple(tu)
    df = pd.DataFrame.from_records(l)
    column_names = [row[0] for row in c.description if (row[0] != 'HOST_ABOUT' and row[0] != 'COMMENTS')]
    columns = [{"id" : i, "name" : column_names[i]} for i in range(len(column_names))]
    print(columns)
    results.append(df.to_dict('records'))
    results.append(columns)

['https://www.airbnb.com/users/show/47267274', 'melissa', datetime.datetime(2015, 10, 23, 0, 0), None, None, 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_small', 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_x_medium', 7, 3]
[('https://www.airbnb.com/users/show/47267274', 'melissa', datetime.datetime(2015, 10, 23, 0, 0), None, None, 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_small', 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_x_medium', 7, 3)]
a
('https://www.airbnb.com/users/show/47267274', 'melissa', datetime.datetime(2015, 10, 23, 0, 0), None, None, 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_small', 'https://a0.muscache.com/im/pictures/e5a05656-cc69-4911-a2e7-ccabc5447162.jpg?aki_policy=profile_x_medium', 7, 3)
[{'id': 0, '

IndexError: list index out of range